## mmaction2 implementation
Only works on linux - for windows use full implementation

In [ ]:
mmadir='/home/administrator/Z/Algorithms/mmaction2/'
src_dir=mmadir+'data/INCAR/rawframes/'#'C:/Users/nmc_costa/Desktop/Desk/mmaction2/data/kinetics_INCAR/rawframes/'
out_dir=mmadir+'data/INCAR/videos/'#'C:/Users/nmc_costa/Desktop/Desk/mmaction2/data/kinetics_INCAR/videos/'
print(src_dir)
print(out_dir)

In [ ]:
!which python

In [ ]:
mma_build_vids_dir=mmadir+'tools/data/' #"C:\Users\nmc_costa\Desktop\Desk\mmaction2\tools\data"

In [ ]:
%cd $mma_build_vids_dir

In [ ]:
!python build_videos.py $src_dir $out_dir --fps 30 --level 2 --num-worker 8 --in-format png --start-idx 0 --filename-tmpl img_%05d --ext mp4

## Full implementation

In [ ]:
src_dir='C:/Users/nmc_costa/Desktop/Desk/mmaction2/data/kinetics_INCAR/rawframes/'
out_dir='C:/Users/nmc_costa/Desktop/Desk/mmaction2/data/kinetics_INCAR/videos/'
print(src_dir)
print(out_dir)

In [ ]:
import argparse
import glob
import os
import os.path as osp
import sys
from multiprocessing import Pool
import multiprocessing, logging

FORMAT = '%(asctime)-15s %(clientip)s %(user)-8s %(message)s'
logging.basicConfig(format=FORMAT)
mpl = multiprocessing.log_to_stderr()
mpl.setLevel(logging.INFO)

In [ ]:
def encode_video(frame_dir_item):
    """Encode frames to video using ffmpeg.

    Args:
        frame_dir_item (list): Rawframe item containing raw frame directory
            full path, rawframe directory (short) path, rawframe directory id.

    Returns:
        bool: Whether synthesize video successfully.
    """
    print("start encoding...")
    full_path, frame_dir_path, frame_dir_id = frame_dir_item
    out_full_path = args.out_dir

    img_name_tmpl = args.filename_tmpl + '.' + args.in_format
    img_path = osp.join(full_path, img_name_tmpl)

    out_vid_name = frame_dir_path + '.' + args.ext
    out_vid_path = osp.join(out_full_path, out_vid_name)
    
    print('img_path:', img_path)
    print('out_vid_path:', out_vid_path)

    cmd = osp.join(
        f"ffmpeg -start_number {args.start_idx} -r {args.fps} -i '{img_path}' "
        f"-vcodec {args.vcodec} '{out_vid_path}'")
    os.system(cmd)

    print(f'{frame_dir_id} {frame_dir_path} done')
    sys.stdout.flush()
    return True

In [ ]:
class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
args = {"level": 2, 
        "fps": 30, 
        "num_worker": 8, 
        "in_format": 'png', 
        "start_idx": 1,
        "filename_tmpl": 'img_%05d', 
        "vcodec":'mpeg4',
        "ext":'mp4',
        "resume":False
       } 
args["src_dir"]=src_dir
args["out_dir"]=out_dir
args = Namespace(**args)

In [ ]:
if not osp.isdir(args.out_dir):
    print(f'Creating folder: {args.out_dir}')
    os.makedirs(args.out_dir)

if args.level == 2:
    classes = os.listdir(args.src_dir)
    for classname in classes:
        new_dir = osp.join(args.out_dir, classname)
        if not osp.isdir(new_dir):
            print(f'Creating folder: {new_dir}')
            os.makedirs(new_dir)

print('Reading rgb frames from folder: ', args.src_dir)
print('Input format of rgb frames: ', args.in_format)
fullpath_list = glob.glob(args.src_dir + '/*' * args.level)
done_fullpath_list = glob.glob(args.src_dir + '/*' * args.level + '.' +
                               args.ext)
print('Total number of rgb frame folders found: ', len(fullpath_list))

if args.resume:
    fullpath_list = set(fullpath_list).difference(set(done_fullpath_list))
    fullpath_list = list(fullpath_list)
    print('Resuming. number of videos to be synthesized: ',
          len(fullpath_list))

if args.level == 2:
    frame_dir_list = list(
        map(
            lambda p: osp.join(
                osp.basename(osp.dirname(p)), osp.basename(p)),
            fullpath_list))
elif args.level == 1:
    frame_dir_list = list(map(lambda p: osp.basename(p), fullpath_list))

print("frame_dir_list:", len(frame_dir_list) )
print("fullpath_list:", len(fullpath_list) )
print("start pool...")

method="for"
if method=="multiprocessing": #works in linux
    pool = Pool(args.num_worker)
    pool.map(encode_video,
             zip(fullpath_list, frame_dir_list, range(len(frame_dir_list))))
if method=="for":
    for i in range(len(frame_dir_list)):
        frame_dir_item = fullpath_list[i], frame_dir_list[i] , i
        encode_video( frame_dir_item )
        